<a href="https://colab.research.google.com/github/fabricecordelieres/Colab_Simple-batch-segmentation-using-CellPose/blob/main/Colab_Simple_batch_segmentation_using_CellPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simple batch segmentation using CellPose**

*This notebook is partly inspired from https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/Cellpose_cell_segmentation_2D_prediction_only.ipynb*

*It allows batch segmentation of 2D/1 channel images, packed as a zip file.
The following default parameters are used:*
* model_type='cyto'
* diameter=100
* flow_threshold=0.3
* cellprob_threshold=-1

*These parameters are expossed as class attributes and can be modified once a "segment_data" object has been initialized (see Step 1.2 for the class definition).*





# Step 0: Before starting
1. Go to Runtime -> Change Runtime Type -> Hardware accelerator.
2. Select "GPU".
3. Press "Save".



# Step 1: Prepare the environment for execution


_Simply execute the following cells and don't worry about the outputs_

In [ ]:
#@markdown ## **_Step 1.1: Install required libraries_**
#@markdown Performs some magic, installing everything that is required for segmentation

#@markdown _Simply execute the following cell and don't worry about the outputs_

!pip install cellpose
!pip install torch torchvision torchaudio

#Fix opencv error: ImportError: cannot import name '_registerMatType' from 'cv2.cv2'
!pip install "opencv-python-headless<4.3"
#exit(0) #Restart Runtime to use newly installed numpy

#Used for progress bars
!pip install tqdm

In [ ]:
#@markdown ## **_Step 1.2: Declare data upload/download workflows_**
#@markdown Defines procedures to upload the data from the disk and pack/download analyzed data as a zip file

#@markdown _Simply execute the following cell_

from google.colab import files
from zipfile import ZipFile
import shutil
import os
import tqdm
import numpy as np
import skimage.io
from cellpose import models,core, io
import sys

#-----------------UPLOAD DATA, PERFORM SEGMENTATION AND DOWNLOAD THE RESULTS-----------------
class segment_data():
  """
  1-Uploads a zip file
  2-Unzips it
  3-Performs the segmentation on all imagess
  4-Zips the results
  5-Downloads the output zip file

  Limitation: only works on one channel images
  """

  #Class attibute
  uploaded=''
  in_dir=''
  in_file=''
  out_dir='/content/_segmentedData/'
  out_file='/content/_segmentedData.zip'

  #Cell pose parameters
  model_type='cyto'#Possible models: ["cyto", "nuclei", "cyto2", "tissuenet", "livecell"]
  segment_channel=0
  diameter=100
  use_GPU=True
  channels=[segment_channel,0]

  flow_threshold=0.3
  cellprob_threshold=-1

  def __init__(self):
    '''Uploads the input zip file and keeps track of it as a class attribute'''
    #Clean anything that might have been generated in a previous run
    self.clean_up_ze_mess()

    #Display infos about the hardware
    if core.use_gpu()==False:
      #Warnings from the ZeroCost StarDist notebook
      print('No access to a GPU: did you follow step 1.1 ?\n If so, try reconnecting later for a more efficient processing or proceed using only CPU\n')
      self.use_GPU=False
    else:
      print("You have access to the GPU.\n")
      print("*************************************************")
      !nvidia-smi
      self.use_GPU=True

    #File chooser
    self.uploaded = files.upload()

  def generate_output(self):
    '''Unzips the uploaded file, performs segmentations, creates the output zip file and downloads it'''
    #Unzip file(s), segments/downloads/clean up
    for filename in self.uploaded.keys():
      #Create a folder named after the file
      zip_extract_path='/content/'+filename.replace('.zip', '')
      os.mkdir(zip_extract_path)

      #Extract the file
      zip_in=ZipFile(filename, 'r')
      zip_in.extractall(zip_extract_path)
      zip_in.close()

      #Set all the folders for input/output
      self.in_file='/content/'+filename
      self.in_dir=self.in_file.replace('.zip', '/')
      if not os.path.exists(self.out_dir):
        os.mkdir(self.out_dir)

      imgList=os.listdir(self.in_dir)
      zip_out = ZipFile(self.out_file, "w")

      model = models.Cellpose(gpu=self.use_GPU, model_type=self.model_type)

      #Segment and add to output zip file
      #for fileName in imgList:
      for fileName in (pbar :=tqdm.tqdm(imgList)):
        pbar.set_description("Processing %s" % fileName)
        img=io.imread(os.path.join(self.in_dir, fileName))
        mask, flow, style, diam = model.eval(img, diameter=self.diameter, flow_threshold=self.flow_threshold,cellprob_threshold=self.cellprob_threshold, channels=self.channels)

        #Output name for masks
        mask_output_name=self.out_dir+fileName
        #Save mask as 16-bit in case this has to be used for detecting than 255 objects
        mask=mask.astype(np.uint32)
        #Save the mask
        skimage.io.imsave(mask_output_name, mask, check_contrast=False)

        zip_out.write(mask_output_name, fileName)


      #Save a text file containing the parameters used for cellpose
      with open(self.out_dir+'params.txt', 'w') as f:
          f.write('model_type: '+self.model_type+'\n')
          f.write('diameter: '+str(self.diameter)+'\n')
          f.write('use_GPU: '+str(self.use_GPU)+'\n')
          f.write('flow_threshold: '+str(self.flow_threshold)+'\n')
          f.write('cellprob_threshold: '+str(self.cellprob_threshold))

      zip_out.write(self.out_dir+'params.txt', 'params.txt')

      #Close the zip file
      zip_out.close()

      files.download(self.out_file)
      self.clean_up_ze_mess()

  def clean_up_ze_mess(self):
    '''Removes the input zip file and the extracted folders'''
    if(os.path.exists('/content/__MACOSX')):
      shutil.rmtree('/content/__MACOSX')
    if(os.path.exists(self.in_dir)):
      shutil.rmtree(self.in_dir)
    if(os.path.exists(self.out_dir)):
      shutil.rmtree(self.out_dir)
    if(os.path.exists(self.in_file)):
      os.remove(self.in_file)

  def delete_output_file(self):
    '''Deletes the output zip file: DO NOT CALL IT BEFORE DOWNLOAD HAS ENDED'''
    self.clean_up_ze_mess()
    os.remove(self.out_file)

# Step 2: Perform the actual segmentation

In [ ]:

#@markdown ## **_Step 2.1: Upload data, run segmentation and download results_**

#@markdown _Simply execute the following cell_

#@markdown 1. Some information will be displayed about the ressources available. In case you see the "no GPU" message, follow the instructions.
#@markdown 2. You'll get asked to select a file containing all your images as a zip file: it may take some time to get your data uploaded, but once set, you won't have to wait for the transfer to complete !
#@markdown 3. Have a break whilst Google is working for you. You may have a look at the progress bar to check everything is running smoothly.
#@markdown 4. The segmented images will be packed into a zip file that will be automatically downloaded to your computer.

segment_it=segment_data()
segment_it.generate_output()

In [ ]:
#@markdown ## **_Step 2.2: Clean the drive_**

#@markdown Erases all files and folder uploaded/generated during segmentation

#@markdown ## <font color="red">**_DO NOT RUN THIS BEFORE YOU'VE CHECKED THE DOWNLOAD IS COMPLETED !_**</font>

#@markdown _Simply execute the following cell_

segment_it.delete_output_file()